In [1]:
import collections
import itertools
from functools import cache

In [2]:
testlines = '''029A
980A
179A
456A
379A'''.splitlines()

In [3]:
data = '''671A
279A
083A
974A
386A'''.splitlines()

## Part 1 ##

In [4]:
numpad = {'A': {'0': '<', '3': '^'},
           '0': {'2': '^', 'A': '>'},
           '1': {'2': '>', '4': '^'},
           '2': {'0': 'v', '3': '>', '5': '^', '1': '<'},
           '3': {'A': 'v', '2': '<', '6': '^'},
           '4': {'1': 'v', '5': '>', '7': '^'},
           '5': {'2': 'v', '6': '>', '8': '^', '4': '<'},
           '6': {'3': 'v', '9': '^', '5': '<'},
           '7': {'4': 'v', '8': '>'},
           '8': {'5': 'v', '9': '>', '7': '<'},
           '9': {'6': 'v', '8': '<'},
          }

In [5]:
dirpad = {'A': {'>': 'v', '^': '<'},
          '^': {'A': '>', 'v': 'v'},
          '>': {'A': '^', 'v': '<'},
          'v': {'>': '>', '^': '^', '<': '<'},
          '<': {'v': '>'},
         }

In [6]:
def get_path(graph, start, end):
    if start == end:
        return ''
    frontier = collections.deque()
    frontier.append(start)
    came_from = {}
    came_from[start] = None
    while frontier:
        current = frontier.popleft()
        for adj in graph[current]:
            if adj not in came_from:
                frontier.append(adj)
                came_from[adj] = current
    prev = end
    current = came_from[end]
    path = [graph[current][prev]]
    while current != start:
        prev = current
        current = came_from[prev]
        path.append(graph[current][prev])
    path.reverse()
    return ''.join(path)

In [7]:
def moves_for_numpad_presses(numpad_presses):
    paths = []
    for start, end in itertools.pairwise(''.join(['A', numpad_presses])):
        paths.append(get_path(numpad, start, end))
    return 'A'.join(paths)+'A'

In [8]:
for seq in testlines:
    print(seq, ': ', moves_for_numpad_presses(seq))

029A :  <A^A>^^AvvvA
980A :  ^^^A<AvvvA>A
179A :  <^<A^^A>>AvvvA
456A :  <^^<A>A>AvvA
379A :  ^A<^^<A>>AvvvA


In [9]:
def dirpad_for_dirpad_presses(seq):
    paths = []
    for start, end in itertools.pairwise(''.join(['A', seq])):
        paths.append(get_path(dirpad, start, end))
    return 'A'.join(paths)+'A'

In [10]:
dirpad_for_dirpad_presses('<A^A>^^AvvvA')

'v<<A>>^A<A>AvA^<AA>Av<AAA>^A'

In [11]:
len(dirpad_for_dirpad_presses('v<<A>>^A<A>AvA^<AA>Av<AAA>^A'))

68

In [12]:
moves_for_numpad_presses('179A')

'<^<A^^A>>AvvvA'

In [13]:
dirpad_for_dirpad_presses(moves_for_numpad_presses('179A'))

'v<<A>^Av<A>>^A<AA>AvAA^Av<AAA>^A'

In [14]:
len(dirpad_for_dirpad_presses(dirpad_for_dirpad_presses(moves_for_numpad_presses('179A'))))

76

That's not right!  Shortest overall path needs to optimize between dirpads. Paths that lead to repeated directions or As is best. It's a multi-level pathing problem.
Let's borrow from https://www.reddit.com/r/adventofcode/comments/1hj2odw/comment/m4ui1sy/

In [15]:
NUMERIC = '789', '456', '123', ' 0A'
DIRECTIONAL = ' ^A', '<v>'

def walk(keypad, x, y, path):
    for direction in path:
        neighbors = (x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)
        x, y = neighbors['<>^v'.index(direction)]
        yield keypad[y][x]

def paths_between(keypad, start, end):
    x1, y1 = next((x, y) for y, row in enumerate(keypad)
                         for x, key in enumerate(row) if key == start)
    x2, y2 = next((x, y) for y, row in enumerate(keypad)
                         for x, key in enumerate(row) if key == end)
    hor = '<>'[x2 > x1] * abs(x2 - x1)
    ver = '^v'[y2 > y1] * abs(y2 - y1)
    return tuple(path + 'A' for path in {hor + ver, ver + hor}
                 if ' ' not in walk(keypad, x1, y1, path))

Shortest path for downstream keypads will always correspond to repeated keypresses. So, L-shaped movements: up or down, followed by left or right, or vice versa.
If one of the L's goes through the gap, then it's not a valid solution.  So lots of cases of only one path, but some will have 2.

In [16]:
paths_between(NUMERIC, 'A', '5') # two valid L's

('^^<A', '<^^A')

In [17]:
paths_between(DIRECTIONAL, 'A', '<') # only one valid L, down then over, since over then down would go through the gap.

('v<<A',)

I would not have come up with the following, which is a memoized, recursive solution.

In [18]:
@cache
def cost_between(keypad, start, end, links):
    return min(cost(DIRECTIONAL, path, links - 1)
               for path in paths_between(keypad, start, end)) if links else 1

@cache
def cost(keypad, keys, links):
    return sum(cost_between(keypad, a, b, links)
               for a, b in itertools.pairwise('A' + keys))

In [19]:
cost(NUMERIC, '179A', 3) # gives the correct answer, not my incorrect one.

68

In [20]:
cost(NUMERIC, '179A', 26)

81251039228

In [21]:
def complexity(code, robots):
    return cost(NUMERIC, code, robots + 1) * int(code[:-1])

print(sum(complexity(code, 2) for code in data))
print(sum(complexity(code, 25) for code in data))

171596
209268004868246
